In [11]:
from splunk_functions import splunk_query
from sourcetypes import sourcetypes
import os
from datetime import datetime
import requests
import json

In [12]:
SPLUNK_HOST="localhost"
SPLUNK_PORT="8089"
username=os.getenv('SPLUNK_UN')
password=os.getenv('SPLUNK_PW')


In [13]:
# The REST API endpoint URL
url = f'https://{SPLUNK_HOST}:{SPLUNK_PORT}/services/data/indexes?output_mode=json'

# Make a GET request to the Splunk REST API
response = requests.get(url, auth=(username, password), verify=False)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract the index names
    index_names = [entry['name'] for entry in data['entry']]

indices="\n".join(index_names)

/home/bots/BOTS/bots/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [14]:
index='botsv2'

In [15]:
source_types_list = sourcetypes.split(" ")

In [16]:
sourcetype = source_types_list[0]

In [17]:
splunk_data_dict = {}

In [18]:
for sourcetype in source_types_list:
    fields = splunk_query(f"index={index} sourcetype={sourcetype}| fieldsummary | stats values(field)")
    fields_list = [x for x in fields.split(" ") if x != ""][1:]
    
    splunk_data_dict[sourcetype] = fields_list

In [19]:
splunk_json = json.dumps(splunk_data_dict)

with open(f"{index}_splunk_field.json", "w") as f:
    f.write(splunk_json)